# Inference PointCloud MMDetection3D

In [2]:
import pickle
import os

import open3d as o3d
import numpy as np
from open3d.web_visualizer import draw
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import mmdet3d

from mmdet3d.apis import inference_detector, init_model
# from sensus.utils.data_converter import pc2pc_object
from mmdet3d.utils import register_all_modules




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


# KITTI Dataset and MODEL

In [4]:
import numpy as np
from mmdet3d.structures.points import get_points_type
from mmdet3d.datasets.transforms.loading import LoadPointsFromFile

def pc2pc_object(pc, pipeline):
    pipeline_dict = {**pipeline[0]}
    assert pipeline_dict['type'] == 'LoadPointsFromFile'
    pipeline_dict.pop('type')
    pc_loader = LoadPointsFromFile(**pipeline_dict)

    pc = pc.reshape(-1, pc_loader.load_dim)
    pc = pc[:, pc_loader.use_dim]
    if pc_loader.norm_intensity:
            assert len(pc_loader.use_dim) >= 4, \
                f'When using intensity norm, expect used dimensions >= 4, got {len(pc_loader.use_dim)}'  # noqa: E501
            pc[:, 3] = np.tanh(pc[:, 3])
    attribute_dims = None

    if pc_loader.shift_height:
        floor_height = np.percentile(pc[:, 2], 0.99)
        height = pc[:, 2] - floor_height
        pc = np.concatenate(
            [pc[:, :3],
                np.expand_dims(height, 1), pc[:, 3:]], 1)
        attribute_dims = dict(height=3)

    if pc_loader.use_color:
        assert len(pc_loader.use_dim) >= 6
        if attribute_dims is None:
            attribute_dims = dict()
        attribute_dims.update(
            dict(color=[
                pc.shape[1] - 3,
                pc.shape[1] - 2,
                pc.shape[1] - 1,
            ]))

    points_class = get_points_type(pc_loader.coord_type)
    pc_object = points_class(
        pc, points_dim=pc.shape[-1], attribute_dims=attribute_dims)

    return pc_object, pc

In [5]:
register_all_modules()

## Build the model from a config file and a checkpoint file
config_file = '../../mmdetection3d/configs/second/second_hv_secfpn_8xb6-80e_kitti-3d-car.py'
checkpoint_file = '../../mmdetection3d/checkpoints/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth'
pcd_path = 'kitti/001856.bin'
model = init_model(config_file, checkpoint_file, device='cuda:0')
result, data = inference_detector(model, pcd_path)

/home/javier/sensus-loci/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:92: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


Loads checkpoint by local backend from path: ../../mmdetection3d/checkpoints/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth


/home/javier/miniconda3/envs/openmmlab/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552411/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
register_all_modules()

## Build the model from a config file and a checkpoint file
config_file = '../configs/dair/main_config.py'
checkpoint_file = '../../work_dirs/main_config/epoch_20.pth'
config_file = '../../mmdetection3d/configs/second/second_hv_secfpn_8xb6-80e_kitti-3d-car.py'
checkpoint_file = '../../mmdetection3d/checkpoints/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth'
pcd_path = 'DAIR/000000/000000.bin'
model = init_model(config_file, checkpoint_file, device='cuda:0')
result, data = inference_detector(model, pcd_path)


Loads checkpoint by local backend from path: ../../mmdetection3d/checkpoints/hv_second_secfpn_6x8_80e_kitti-3d-car_20200620_230238-393f000c.pth


In [8]:
print(result.pred_instances_3d.bboxes_3d.tensor)

tensor([[ 37.2186,  -7.9195,  -1.5823,   4.1206,   1.8063,   1.4754,   1.6178],
        [ 20.2096,  -3.8007,  -1.5950,   3.9469,   1.6521,   1.4481,   1.5821],
        [ 44.7978, -31.1231,  -2.0921,   4.2525,   1.7916,   1.8427,   3.2053],
        [ 44.7518,  16.9505,  -1.8428,   4.5110,   1.9621,   1.8811,  -1.5078],
        [ 29.9435,  13.4715,  -1.5503,   4.2660,   1.9138,   1.5197,  -1.4848],
        [ 12.8899,  -7.0814,  -1.4948,   3.9369,   1.6347,   1.4288,   4.7064],
        [ 48.5143,  19.9207,  -1.6766,   4.3103,   1.8303,   1.8755,  -1.5320],
        [ 37.3584,  -1.4663,  -1.5080,   4.0174,   1.5957,   1.4266,   1.7044],
        [ 52.0175,  -4.9251,  -1.6035,   4.0532,   1.6913,   1.5623,  -1.5417],
        [ 24.2077,  -7.7865,  -1.6153,   4.1451,   1.7726,   1.4754,  -1.5417],
        [ 39.7922,  26.6046,  -2.0245,   4.4427,   1.9151,   1.7010,  -1.4527],
        [ 60.0520,  12.4946,  -1.5971,   4.4551,   1.8914,   2.0473,  -1.5474],
        [ 56.7848,  25.2805,  -2.2197,  

## Lets draw

In [9]:
def generate_bbox(bbox_tensor):
    # Extract label info
    dimensions = bbox_tensor[3:6]
    position = bbox_tensor[0:3]
    rotation = bbox_tensor[6]

    print(dimensions)
    
    # Generate bbox
    box = o3d.geometry.TriangleMesh.create_box(width=dimensions[0], height=dimensions[1], depth=dimensions[2])

    # Width is x, height is -y and z is up
    box.paint_uniform_color([1.0, 0.0, 0.0]) # Set color to red

    # Translate box
    box.translate(position)

    # Translate box to center
    box.translate([-dimensions[0]/2, -dimensions[1]/2, 0])

    # Rotate box
    center = box.get_center()
    rotation = box.get_rotation_matrix_from_xyz((0, 0, -rotation - np.pi/2)) # Notice the negative sign for rotation

    box.rotate(rotation, center=center)

    lines = o3d.geometry.LineSet.create_from_triangle_mesh(box)
    # Remove lines that connect non-adjacent points
    lines.lines = o3d.utility.Vector2iVector(np.array([[0, 1], [2, 0], [2, 3], [3, 1], [4, 5], [4, 6], [6, 7], [7, 5], [0, 4], [1, 5], [2, 6], [3, 7]]))
    lines.paint_uniform_color([1, 0, 0])

    return lines

In [10]:
# Read pc from bin file
with open(pcd_path, 'rb') as f:
    points = np.fromfile(f, dtype=np.float32, count=-1).reshape([-1, 4])

pcd_bin = o3d.geometry.PointCloud()
pcd_bin.points = o3d.utility.Vector3dVector(points[:, :3])
print(pcd_bin)

PointCloud with 56285 points.


In [11]:
bboxes3d = result.pred_instances_3d.bboxes_3d.tensor.to('cpu').detach().numpy()

bbox = generate_bbox(bboxes3d[0])

[4.1206164 1.8063418 1.475353 ]


In [13]:
# # Generate bbox
# lines = generate_bbox(label, calib)

# Draw bbox into point cloud
draw([pcd_bin, bbox], width=900, height=600, point_size=2)

[Open3D INFO] Window window_1 created.


WebVisualizer(window_uid='window_1')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.102151676384729
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.102151676384729
[Open3D INFO] Sending init frames to window_1.


[000:000][682631] (stun_port.cc:96): Binding request timed out from 163.117.150.x:51524 (enp4s0)
[004:867][682631] (stun_port.cc:96): Binding request timed out from 163.117.150.x:41088 (enp4s0)
